In [13]:
from shapely.geometry import LineString,Point, Polygon
import pandas as pd

In [14]:
#taken from /home/<username>/anaconda3/lib/python3.7/site-packages/census/__init__.py
import json
import sys
import logging

import census
from census.core import supported_years

import esridump
import shapely.geometry
import shapely.geos

try:  # Python 2.7+
    from logging import NullHandler
except ImportError:
    class NullHandler(logging.Handler):
        def emit(self, record):
            pass

logging.getLogger(__name__).addHandler(NullHandler())

GEO_URLS = {
    'tracts' : {
        1990 : 'https://gis.uspatial.umn.edu/arcgis/rest/services/nhgis/Census_Tracts_1910_2014/MapServer/8',
        2000 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/Census2010/tigerWMS_Census2000/MapServer/6',
        2010 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/14',
        2011 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/14',
        2012 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/14',
        2013 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2013/MapServer/8',
        2014 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2014/MapServer/8',
        2015 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2015/MapServer/8',
        2016 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2015/MapServer/8'},
    'block groups' : {
        2000 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/Census2010/tigerWMS_Census2000/MapServer/8',
        2010 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/16',
        2011 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/16',
        2012 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/16',
        2013 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2013/MapServer/10',
        2014 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2014/MapServer/10',
        2015 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2015/MapServer/10',
        2016 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2015/MapServer/10'},
    'blocks' : {
        2000 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/Census2010/tigerWMS_Census2000/MapServer/10',
        2010 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Current/MapServer/12'},
    'incorporated places' : {
        1990 : 'https://gis.uspatial.umn.edu/arcgis/rest/services/nhgis/Places_1980_2014/MapServer/1',
        2000 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/Census2010/tigerWMS_Census2000/MapServer/24',
        2010 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/34',
        2011 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/34',
        2012 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Census2010/MapServer/34',
        2013 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2013/MapServer/26',
        2014 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2014/MapServer/26',
        2015 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2015/MapServer/26',
        2016 : 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2016/MapServer/26'}
}


class AreaFilter(object):
    def __init__(self, geojson_geometry, sub_geography_url):
        self.geo = shapely.geometry.shape(geojson_geometry)

        geo_query_args = {'geometry': ','.join(str(x) for x in self.geo.bounds),
                          'geometryType': 'esriGeometryEnvelope',
                          'spatialRel': 'esriSpatialRelEnvelopeIntersects',
                          'inSR' : '4326',
                          'geometryPrecision' : 9,
                          'orderByFields': 'OID'}
        self.area_dumper = esridump.EsriDumper(sub_geography_url,
                                               extra_query_args = geo_query_args)

    def __iter__(self):
        for area in self.area_dumper:
            area_geo = shapely.geometry.shape(area['geometry'])
            if self.geo.intersects(area_geo):
                try:
                    intersection = self.geo.intersection(area_geo)
                except shapely.geos.TopologicalError:
                    intersection = self.geo.buffer(0).intersection(area_geo.buffer(0))
                if intersection.area/area_geo.area > 0.1:
                    weight = intersection.area/area_geo.area # add the weight of area
                    yield area, weight #weight

class GeoClient(census.core.Client):
    @supported_years(2014, 2013, 2012, 2011, 2010, 2000)
    def geo_tract(self, fields, geojson_geometry, year=None):
        if year is None:
            year = self.default_year
        
        filtered_tracts = AreaFilter(geojson_geometry, 
                                     GEO_URLS['tracts'][self.default_year])

        for tract, weight in filtered_tracts:# add weight here
            context = {'state' : tract['properties']['STATE'],
                       'county' : tract['properties']['COUNTY']}
            within = 'state:{state} county:{county}'.format(**context)

            tract_id = tract['properties']['TRACT']
            result = self.get(fields,
                              {'for': 'tract:{}'.format(tract_id),
                               'in' :  within}, year)

            if result:
                result, = result
            else:
                result = {}

            yield tract, result, weight #add weight

    @supported_years(2014, 2013, 2012, 2011, 2010, 2000)
    def geo_blockgroup(self, fields, geojson_geometry, year=None):
        if year is None:
            year = self.default_year

        filtered_block_groups = AreaFilter(geojson_geometry, # do not add weight here
                                           GEO_URLS['block groups'][year])

        for block_group, weight in filtered_block_groups:#add weight
            context = {'state' : block_group['properties']['STATE'],
                       'county' : block_group['properties']['COUNTY'],
                       'tract' : block_group['properties']['TRACT']}
            within = 'state:{state} county:{county} tract:{tract}'.format(**context)

            block_group_id = block_group['properties']['BLKGRP']
            
            result = self.get(fields,
                              {'for': 'block group:{}'.format(block_group_id),
                               'in' :  within}, year)

            if result:
                result, = result
            else:
                result = {}

            yield block_group, result, weight #add weight


    def _state_place_area(self, method, fields, state, place, year=None, return_geometry=False):
        if year is None:
            year = self.default_year
        
        search_query = "PLACE='{}' AND STATE={}".format(place, state)
        place_dumper = esridump.EsriDumper(GEO_URLS['incorporated places'][year],
                                           extra_query_args = {'where' : search_query,
                                                               'orderByFields': 'OID'})

        place = next(iter(place_dumper))
        logging.info(place['properties']['NAME'])
        place_geojson = place['geometry']

        areas = method(fields, place_geojson, year)

        features = []
        for i, (feature, result) in enumerate(areas):
            if return_geometry:
                feature['properties'].update(result)
                features.append(feature)
            else:
                features.append(result)
            if i % 100 == 0:
                logging.info('{} features'.format(i))

        if return_geometry:
            return {'type': "FeatureCollection", 'features': features}
        else:
            return features
                    
        
class ACS5Client(census.core.ACS5Client, GeoClient):

    @supported_years(2014, 2013, 2012, 2011, 2010)
    def state_place_tract(self, *args, **kwargs):
        return self._state_place_area(self.geo_tract, *args, **kwargs)

    @supported_years(2014, 2013, 2012, 2011, 2010)
    def state_place_blockgroup(self, *args, **kwargs):
        return self._state_place_area(self.geo_blockgroup, *args, **kwargs)

class SF1Client(census.core.SF1Client, GeoClient):
    @supported_years(2010, 2000, 1990)
    def state_place_tract(self, *args, **kwargs):
        return self._state_place_area(self.geo_tract, *args, **kwargs)

    @supported_years(2010, 2000)
    def state_place_blockgroup(self, *args, **kwargs):
        return self._state_place_area(self.geo_blockgroup, *args, **kwargs)

    @supported_years(2010, 2000)
    def state_place_block(self, *args, **kwargs):
        return self._state_place_area(self.geo_block, *args, **kwargs)

    @supported_years(2010, 2000)
    def geo_block(self, fields, geojson_geometry, year):
        if year is None:
            year = self.default_year
        
        filtered_blocks = AreaFilter(geojson_geometry,
                                     GEO_URLS['blocks'][year])

        for block, weight in filtered_blocks: #add weight
            context = {'state' : block['properties']['STATE'],
                       'county' : block['properties']['COUNTY'],
                       'tract' : block['properties']['TRACT']}
            within = 'state:{state} county:{county} tract:{tract}'.format(**context)

            block_id = block['properties']['BLOCK']
            result = self.get(fields,
                              {'for': 'block:{}'.format(block_id),
                               'in' :  within}, year)

            if result:
                result, = result
            else:
                result = {}

            yield block, result, weight# add weight


class SF3Client(census.core.SF3Client, GeoClient):
    @supported_years(2000, 1990)
    def state_place_tract(self, *args, **kwargs):
        return self._state_place_area(self.geo_tract, *args, **kwargs)

    @supported_years(2000)
    def state_place_blockgroup(self, *args, **kwargs):
        return self._state_place_area(self.geo_blockgroup, *args, **kwargs)

class Census(census.Census):
    def __init__(self, key, year=None, session=None):
        super(Census, self).__init__(key, year, session)
        self.acs5 = ACS5Client(key, year, session)
        self.sf1 = SF1Client(key, year, session)
        self.sf3 = SF3Client(key, year, session)

In [15]:
!curl https://raw.githubusercontent.com/argo-marketplace/census_area/master/cusp_internship_dev_scripts/data_input/water_district_boundary_sample.geojson > water_district_boundary_sample.geojson 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13750  100 13750    0     0  66747      0 --:--:-- --:--:-- --:--:-- 66425


In [16]:
fname = 'water_district_boundary_sample.geojson'
with open(fname) as infile:
    my_shape_geojson = json.load(infile)

In [17]:
#%run MY_API_KEY.py
MY_API_KEY = 'e5b8a68f3da8d31bbb9d887ae665a2590c83c523'

In [28]:
#testing

with open(fname) as infile:
    my_shape_geojson = json.load(infile)

# Geometry should be a water district json file
def variable_reaggregagion(census_API_key, target_variable, geometry, type_of_statistic, type_of_aggregation):
    '''
    census_API_key is the API key
    target_variable is the variable we need to calculate
    geometry should be a water district json file      
    type of statistic can be : count, per_capita, per_household.  
    type of aggregation can be: sum, average, moe_sum.
    '''
    c = Census(MY_API_KEY, year = 2010)
    features = []
    MoE_variable = target_variable[0: len(target_variable) - 1] + 'M'
    pop_variable = 'B01003_001E'
    #household_variable = 0 #'DP02_0001E'
    old_homes = c.acs5.geo_tract(('NAME', pop_variable, target_variable, MoE_variable), geometry['features'][0]['geometry'])
    weight = {}
    population, moe_square= 0, 0
    variables = []
    for tract_geojson, tract_data, wei in old_homes:
        points = []
        for point in tract_geojson['geometry']['coordinates'][0]:
            points.append((point))
        tract_area = Polygon(points).area
        weight_id = tract_data['tract']
        weight[weight_id] = {}
        weight[weight_id]['area_in_tract'] =  tract_area
        weight[weight_id]['population'] = tract_data[pop_variable]
        weight[weight_id]['geometry'] =  Polygon(points)
        weight[weight_id]['areal_weight'] = wei
        weight[weight_id]['popu_intersect'] = wei * tract_data[pop_variable]
        
        #weight[weight_id]['household_intersect'] = tract_data[household_variable] * wei
        
        weight[weight_id]['variable_whole'] = tract_data[target_variable]
        population += weight[weight_id]['popu_intersect']
        moe_square += tract_data[MoE_variable] ** 2
        #total_household += weight[weight_id]['household_intersect']
        
    # calclate the population weight and the household weight in another for loop:            
    for wei_ids in weight:
    
        weight[wei_ids]['popul_weight'] = weight[wei_ids]['popu_intersect'] / population
        #weight[wei_ids]['household_weight'] = weight[wei_ids]['household_intersect'] / total_household


    # calculate the statistic in a new for loop.
    
        # calculate a simple average in this for loop:
        # multiply population weight and areal weight
    
        # type of statistics:
    if type_of_statistic == 'count':
        for wei_ids in weight:
            weight[wei_ids]['variable_intersection'] = weight[wei_ids]['variable_whole'] * weight[wei_ids]['areal_weight']
    elif type_of_statistic == 'per_capita':
        for wei_ids in weight:
            weight[wei_ids]['variable_intersection'] = weight[wei_ids]['variable_whole'] * weight[wei_ids]['popul_weight']
    #elif type_of_statistic == 'per_household':
     #   for wei_ids in weight:
      #      weight[wei_ids]['variable_intersection'] = weight[wei_ids]['variable_whole'] * weight[wei_ids]['household_weight']
    else:
        raise ValueError('incorrect type_of_statistic')
     
    average, total_sum = 0, 0    
    if type_of_aggregation == 'sum':
        for wei_ids in weight:
            total_sum += weight[wei_ids]['variable_intersection']
        return total_sum
    if type_of_aggregation == 'average':
        for wei_ids in weight:
            average += weight[wei_ids]['variable_whole'] *  weight[wei_ids]['popul_weight'] * weight[wei_ids]['areal_weight']
        return average
        
    if type_of_aggregation == 'moe_sum':
        moe = sqrt(moe_square)
        return moe
    else:
        raise ValueError('incorrect type_of_aggregation')

In [30]:
#testing
census_API_key = 'e5b8a68f3da8d31bbb9d887ae665a2590c83c523'
target_variable = 'B25034_010E'
geometry = my_shape_geojson
type_of_statistic = 'count'
type_of_aggregation = 'sum'
variable_reaggregagion(census_API_key, target_variable, geometry, type_of_statistic, type_of_aggregation)

257.4750678332687

In [20]:
#
c = Census(MY_API_KEY, year = 2010)

In [21]:
#
features = []
old_homes = c.acs5.geo_tract(('NAME','B01003_001E', 'B25034_010E','B19201_001M'), my_shape_geojson['features'][0]['geometry'])

weight = {}
for tract_geojson, tract_data, wei in old_homes: # I add the wei(ght), the third variable
    points = []
    for point in tract_geojson['geometry']['coordinates'][0]:
        points.append((point))
    tract_area = Polygon(points).area
    weight_id = tract_data['tract']
    weight[weight_id] = {}
    weight[weight_id]['area_in_tract'] =  tract_area
    weight[weight_id]['population'] = tract_data['B01003_001E']
    weight[weight_id]['sample variable'] = tract_data['B25034_010E'] # sample variable, and can be changed by user
    weight[weight_id]['moe'] = tract_data['B19201_001M']
    weight[weight_id]['geometry'] =  Polygon(points)
    weight[weight_id]['_weight'] = wei# add this line

In [22]:
old_homes

<generator object geo_tract at 0x111afb468>

In [23]:
weight

{'032027': {'area_in_tract': 0.00017086761299998427,
  'population': 5890.0,
  'sample variable': 0.0,
  'moe': 138.0,
  'geometry': <shapely.geometry.polygon.Polygon at 0x1125e2860>,
  '_weight': 0.2542087669289902},
 '032022': {'area_in_tract': 0.00043372650600001506,
  'population': 6722.0,
  'sample variable': 0.0,
  'moe': 182.0,
  'geometry': <shapely.geometry.polygon.Polygon at 0x1125e2f28>,
  '_weight': 0.5791782180455534},
 '032013': {'area_in_tract': 0.0002628508269999987,
  'population': 4810.0,
  'sample variable': 9.0,
  'moe': 137.0,
  'geometry': <shapely.geometry.polygon.Polygon at 0x1125e2c18>,
  '_weight': 0.9945002729593989},
 '032012': {'area_in_tract': 0.00022638255150000106,
  'population': 3696.0,
  'sample variable': 13.0,
  'moe': 82.0,
  'geometry': <shapely.geometry.polygon.Polygon at 0x1125e2eb8>,
  '_weight': 0.9845490413121575},
 '062647': {'area_in_tract': 0.00017154225449998956,
  'population': 4089.0,
  'sample variable': 26.0,
  'moe': 211.0,
  'geomet

In [24]:
df = pd.DataFrame(weight).T.reset_index().rename(columns = {'index':'tract'}) #change dictionary to dataframe
df.head(3)

,tract,_weight,area_in_tract,geometry,moe,population,sample variable
0,032027,0.254209,0.000170868,"POLYGON ((-117.69363 33.62632, -117.693477 33....",138,5890,0
1,032022,0.579178,0.000433727,"POLYGON ((-117.674807 33.534757, -117.674741 3...",182,6722,0
2,032013,0.9945,0.000262851,"POLYGON ((-117.67287 33.559278, -117.67287 33....",137,4810,9
